In [5]:
#conda update smart_open
#!pip3 install tweet-preprocessor --user

  Using cached https://files.pythonhosted.org/packages/17/9d/71bd016a9edcef8860c607e531f30bd09b13103c7951ae73dd2bf174163c/tweet_preprocessor-0.6.0-py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import gensim
#import preprocessor as p
import preprocessor as p
import nltk
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import euclidean_distances
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string

In [11]:
df = pd.read_csv("tweets.csv", usecols=['text', 'id'])
df.head()

,id,text
0,784609194234306560,Here is my statement.pic.twitter.com/WAZiGoQqMQ
1,785608815962099712,Is this really America? Terrible!pic.twitter.c...
2,784840992734064640,The media and establishment want me out of the...
3,784767399442653184,Certainly has been an interesting 24 hours!
4,785561269571026944,Debate polls look great - thank you!\n#MAGA #A...


In [70]:
list_text_df = df["text"].to_list()
list_text_df[:10]

['Here is my statement.pic.twitter.com/WAZiGoQqMQ',
 'Is this really America? Terrible!pic.twitter.com/WiwC61PIFu',
 'The media and establishment want me out of the race so badly -  I WILL NEVER DROP OUT OF THE RACE, WILL NEVER LET MY SUPPORTERS DOWN! #MAGA',
 'Certainly has been an interesting 24 hours!',
 'Debate polls look great - thank you!\n#MAGA #AmericaFirstpic.twitter.com/4peQ3Sswdz',
 'WHAT THEY ARE SAYING ABOUT THE CLINTON CAMPAIGN’S ANTI-CATHOLIC BIGOTRY: \nhttp://bit.ly/2dcbtvkCrooked\xa0',
 'Thank you! #MAGA #AmericaFirstpic.twitter.com/fG313wjlKm',
 'I will be in Cincinnati, Ohio tomorrow night at 7:30pm- join me! #OhioVotesEarly #VoteTrumpPence16 \nTickets: https://www.donaldjtrump.com/schedule/register/cincinnati-oh2/\xa0…pic.twitter.com/XUFuGc4Fg5',
 'Very little pick-up by the dishonest media of incredible information provided by WikiLeaks. So dishonest! Rigged system!',
 'Thank you Florida- a MOVEMENT that has never been seen before and will never be seen again. Lets

In [54]:
def preprocess_tweet(text):

    # Check characters to see if they are in punctuation
    #nopunc = [i for item in text for i in item.split()]
    nopunc = [char for char in text if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    # convert text to lower-case
    nopunc = nopunc.lower()
    # remove pic 
    nopunc = re.sub(r'pic.twitter.com/[\w]*',"", nopunc)
    # remove URLs
    nopunc = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', nopunc)
    nopunc = re.sub(r'http\S+', '', nopunc)
    # remove special caracters
    nopunc = re.sub(r'\W+', ' ', nopunc)
    # remove usernames
    nopunc = re.sub('@[^\s]+', '', nopunc)
    # remove the # in #hashtag
    nopunc = re.sub(r'#([^\s]+)', r'\1', nopunc)
    #remove punctuation
    nopunc = re.sub(r'[^\w\s]','',nopunc)
    # remove words with only one character
    nopunc =  re.sub(r"\b[a-zA-Z]\b", "", nopunc)
    # remove repeated characters
    nopunc = word_tokenize(nopunc)
    # remove stopwords from final word list
    return [word for word in nopunc if word not in stopwords.words('english')]

In [101]:
df_text = df.copy()
df_text.head()

,id,text
0,784609194234306560,Here is my statement.pic.twitter.com/WAZiGoQqMQ
1,785608815962099712,Is this really America? Terrible!pic.twitter.c...
2,784840992734064640,The media and establishment want me out of the...
3,784767399442653184,Certainly has been an interesting 24 hours!
4,785561269571026944,Debate polls look great - thank you!\n#MAGA #A...


In [102]:
df_text["clean_text"] = df_text["text"].apply(lambda x: preprocess_tweet([x]))
df_text.head()

,id,text,clean_text
0,784609194234306560,Here is my statement.pic.twitter.com/WAZiGoQqMQ,[statement]
1,785608815962099712,Is this really America? Terrible!pic.twitter.c...,"[really, america, terrible]"
2,784840992734064640,The media and establishment want me out of the...,"[media, establishment, want, race, badly, neve..."
3,784767399442653184,Certainly has been an interesting 24 hours!,"[certainly, interesting, 24, hours]"
4,785561269571026944,Debate polls look great - thank you!\n#MAGA #A...,"[debate, polls, look, great, thank, maga, amer..."


In [116]:
testdf = df_text.copy()
testdf.head()

,id,text,clean_text
0,784609194234306560,Here is my statement.pic.twitter.com/WAZiGoQqMQ,[statement]
1,785608815962099712,Is this really America? Terrible!pic.twitter.c...,"[really, america, terrible]"
2,784840992734064640,The media and establishment want me out of the...,"[media, establishment, want, race, badly, neve..."
3,784767399442653184,Certainly has been an interesting 24 hours!,"[certainly, interesting, 24, hours]"
4,785561269571026944,Debate polls look great - thank you!\n#MAGA #A...,"[debate, polls, look, great, thank, maga, amer..."


In [119]:
testdf["tokenize_text"] = testdf["clean_text"].apply(lambda x: tokenize_text(x))
testdf.head()

,id,text,clean_text,tokenize_text
0,784609194234306560,Here is my statement.pic.twitter.com/WAZiGoQqMQ,[statement],[[statement]]
1,785608815962099712,Is this really America? Terrible!pic.twitter.c...,"[really, america, terrible]","[[really], [america], [terrible]]"
2,784840992734064640,The media and establishment want me out of the...,"[media, establishment, want, race, badly, neve...","[[media], [establishment], [want], [race], [ba..."
3,784767399442653184,Certainly has been an interesting 24 hours!,"[certainly, interesting, 24, hours]","[[certainly], [interesting], [24], [hours]]"
4,785561269571026944,Debate polls look great - thank you!\n#MAGA #A...,"[debate, polls, look, great, thank, maga, amer...","[[debate], [polls], [look], [great], [thank], ..."


In [118]:
def tokenize_text(text):
    tokenized_doc = []
    for element in text:
        tokenized_doc.append(element.split(','))
    return tokenized_doc

In [128]:
tag_words = [TaggedDocument(d, [i]) for i, d in enumerate(testdf.clean_text)]
tag_words

[TaggedDocument(words=['statement'], tags=[0]),
 TaggedDocument(words=['really', 'america', 'terrible'], tags=[1]),
 TaggedDocument(words=['media', 'establishment', 'want', 'race', 'badly', 'never', 'drop', 'race', 'never', 'let', 'supporters', 'maga'], tags=[2]),
 TaggedDocument(words=['certainly', 'interesting', '24', 'hours'], tags=[3]),
 TaggedDocument(words=['debate', 'polls', 'look', 'great', 'thank', 'maga', 'americafirst'], tags=[4]),
 TaggedDocument(words=['saying', 'clinton', 'campaign', 'anti', 'catholic', 'bigotry'], tags=[5]),
 TaggedDocument(words=['thank', 'maga', 'americafirst'], tags=[6]),
 TaggedDocument(words=['cincinnati', 'ohio', 'tomorrow', 'night', '7', '30pm', 'join', 'ohiovotesearly', 'votetrumppence16', 'tickets'], tags=[7]),
 TaggedDocument(words=['little', 'pick', 'dishonest', 'media', 'incredible', 'information', 'provided', 'wikileaks', 'dishonest', 'rigged', 'system'], tags=[8]),
 TaggedDocument(words=['thank', 'florida', 'movement', 'never', 'seen', 'nev

In [129]:
testmodel = Doc2Vec(tag_words, vector_size=20, window=5, min_count=1, workers=-1, epochs = 20)

In [130]:
len(testmodel.wv.vocab)

17524

In [131]:
txt2 = ["Here is my statement"]

In [141]:
test_result_with_df = testmodel.docvecs.most_similar(positive=[testmodel.infer_vector(txt2)], topn=20)
test_result_with_df

[(9339, 0.8056761026382446),
 (8844, 0.7468898296356201),
 (15208, 0.7347336411476135),
 (9447, 0.7097319960594177),
 (8437, 0.7030950784683228),
 (10486, 0.698501467704773),
 (4567, 0.6960663795471191),
 (8161, 0.6896994709968567),
 (14823, 0.669660747051239),
 (9472, 0.665981650352478),
 (17158, 0.6639419198036194),
 (8343, 0.6635996699333191),
 (17009, 0.659203290939331),
 (14930, 0.6582518815994263),
 (10600, 0.6447194814682007),
 (3439, 0.6438406705856323),
 (9848, 0.6438257098197937),
 (10448, 0.6436060667037964),
 (14358, 0.6362336277961731),
 (16681, 0.6355042457580566)]

In [137]:
def get_tweet(resultsimilarity):
    tweet_similar = []
    for elem in resultsimilarity:
        tweet_similar.append({'id': elem[0],'tweets': df.text[elem[0]],'score': elem[1]})
    print(tweet_similar)

In [138]:
get_tweet(test_result_with_df)

[{'id': 9339, 'tweets': 'President Obama, do not attack Syria. There is no upside and tremendous downside. Save your "powder" for another (and more important) day!', 'score': 0.8056761026382446}, {'id': 8844, 'tweets': 'I don’t know why our allies are so surprised Obama is tapping their phones? Nothing changes!', 'score': 0.7468898296356201}, {'id': 15208, 'tweets': 'Based on the ovation last night from the Letterman @Late_Show audience, I believe it will be hard for Obama to throw $5M down the drain....', 'score': 0.7347336411476135}, {'id': 9447, 'tweets': '"@Joe1sPro: @realDonaldTrump the president of Syria is killing people inhumanly"  But the so called "rebels" may be just as bad (or worse)!', 'score': 0.7097319960594177}, {'id': 8437, 'tweets': "We're coming up on the NEW YEAR-It is really important that, despite so many stupid decisions being made in Washington, we make it BEST EVER", 'score': 0.7030950784683228}, {'id': 10486, 'tweets': '"@charmed112586: @realDonaldTrump i live

In [136]:
tweet_similar = []
for elem in test_result_with_df:
    tweet_similar.append({'id': elem[0],'tweets': df.text[elem[0]],'score': elem[1]})
print(tweet_similar, "\n")

[{'id': 9339, 'tweets': 'President Obama, do not attack Syria. There is no upside and tremendous downside. Save your "powder" for another (and more important) day!', 'score': 0.8056761026382446}, {'id': 8844, 'tweets': 'I don’t know why our allies are so surprised Obama is tapping their phones? Nothing changes!', 'score': 0.7468898296356201}, {'id': 15208, 'tweets': 'Based on the ovation last night from the Letterman @Late_Show audience, I believe it will be hard for Obama to throw $5M down the drain....', 'score': 0.7347336411476135}, {'id': 9447, 'tweets': '"@Joe1sPro: @realDonaldTrump the president of Syria is killing people inhumanly"  But the so called "rebels" may be just as bad (or worse)!', 'score': 0.7097319960594177}, {'id': 8437, 'tweets': "We're coming up on the NEW YEAR-It is really important that, despite so many stupid decisions being made in Washington, we make it BEST EVER", 'score': 0.7030950784683228}, {'id': 10486, 'tweets': '"@charmed112586: @realDonaldTrump i live

In [71]:
text_clean_df = preprocess_tweet(list_text)
text_clean_df

['statement',
 'really',
 'america',
 'terrible',
 'media',
 'establishment',
 'want',
 'race',
 'badly',
 'never',
 'drop',
 'race',
 'never',
 'let',
 'supporters',
 'magacertainly',
 'interesting',
 '24',
 'hours',
 'debate',
 'polls',
 'look',
 'great',
 'thank',
 'maga',
 'americafirst',
 'saying',
 'clinton',
 'campaign',
 'anti',
 'catholic',
 'bigotry',
 'thank',
 'maga',
 'americafirst',
 'cincinnati',
 'ohio',
 'tomorrow',
 'night',
 '7',
 '30pm',
 'join',
 'ohiovotesearly',
 'votetrumppence16',
 'tickets',
 'little',
 'pick',
 'dishonest',
 'media',
 'incredible',
 'information',
 'provided',
 'wikileaks',
 'dishonest',
 'rigged',
 'system',
 'thank',
 'florida',
 'movement',
 'never',
 'seen',
 'never',
 'seen',
 'lets',
 'get',
 'votetrumppence16',
 '11',
 '8',
 'maga',
 'foul',
 'mouthed',
 'sen',
 'john',
 'mccain',
 'begged',
 'support',
 'primary',
 'gave',
 'dropped',
 'locker',
 'room',
 'remarks',
 'disloyal',
 'far',
 'difficult',
 'crooked',
 'hillary',
 'come',
 

In [72]:
tokenized_doc = []
for d in text_clean_df:
    tokenized_doc.append(d.split())

In [73]:
tokenized_doc

[['statement'],
 ['really'],
 ['america'],
 ['terrible'],
 ['media'],
 ['establishment'],
 ['want'],
 ['race'],
 ['badly'],
 ['never'],
 ['drop'],
 ['race'],
 ['never'],
 ['let'],
 ['supporters'],
 ['magacertainly'],
 ['interesting'],
 ['24'],
 ['hours'],
 ['debate'],
 ['polls'],
 ['look'],
 ['great'],
 ['thank'],
 ['maga'],
 ['americafirst'],
 ['saying'],
 ['clinton'],
 ['campaign'],
 ['anti'],
 ['catholic'],
 ['bigotry'],
 ['thank'],
 ['maga'],
 ['americafirst'],
 ['cincinnati'],
 ['ohio'],
 ['tomorrow'],
 ['night'],
 ['7'],
 ['30pm'],
 ['join'],
 ['ohiovotesearly'],
 ['votetrumppence16'],
 ['tickets'],
 ['little'],
 ['pick'],
 ['dishonest'],
 ['media'],
 ['incredible'],
 ['information'],
 ['provided'],
 ['wikileaks'],
 ['dishonest'],
 ['rigged'],
 ['system'],
 ['thank'],
 ['florida'],
 ['movement'],
 ['never'],
 ['seen'],
 ['never'],
 ['seen'],
 ['lets'],
 ['get'],
 ['votetrumppence16'],
 ['11'],
 ['8'],
 ['maga'],
 ['foul'],
 ['mouthed'],
 ['sen'],
 ['john'],
 ['mccain'],
 ['begged

In [75]:
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_doc)]
tagged_data

[TaggedDocument(words=['statement'], tags=[0]),
 TaggedDocument(words=['really'], tags=[1]),
 TaggedDocument(words=['america'], tags=[2]),
 TaggedDocument(words=['terrible'], tags=[3]),
 TaggedDocument(words=['media'], tags=[4]),
 TaggedDocument(words=['establishment'], tags=[5]),
 TaggedDocument(words=['want'], tags=[6]),
 TaggedDocument(words=['race'], tags=[7]),
 TaggedDocument(words=['badly'], tags=[8]),
 TaggedDocument(words=['never'], tags=[9]),
 TaggedDocument(words=['drop'], tags=[10]),
 TaggedDocument(words=['race'], tags=[11]),
 TaggedDocument(words=['never'], tags=[12]),
 TaggedDocument(words=['let'], tags=[13]),
 TaggedDocument(words=['supporters'], tags=[14]),
 TaggedDocument(words=['magacertainly'], tags=[15]),
 TaggedDocument(words=['interesting'], tags=[16]),
 TaggedDocument(words=['24'], tags=[17]),
 TaggedDocument(words=['hours'], tags=[18]),
 TaggedDocument(words=['debate'], tags=[19]),
 TaggedDocument(words=['polls'], tags=[20]),
 TaggedDocument(words=['look'], tags

In [60]:
## Train doc2vec model
#model = Doc2Vec(tagged_data, vector_size=20, window=5, min_count=1, workers=-1, epochs = 20)
model = Doc2Vec(tagged_data, vector_size=20, window=5, min_count=1, workers=-1, epochs = 20)

In [61]:
len(model.wv.vocab)

18620

In [62]:
pickle.dump(model, open('Doc2VecModel2.pkl','wb'))

In [63]:
Doc2VecModel2 = pickle.load(open('Doc2VecModel2.pkl','rb'))

In [64]:
len(Doc2VecModel2.wv.vocab)

18620

In [82]:
def convertStrintoList(Sentences):
    listSentences = [Sentences]
    return listSentences

In [83]:
text = "Here is my statement"
list_text = convertStrintoList(text)
type(list_text)

list

In [84]:
list_text

['Here is my statement']

In [65]:
#txt = "Here is my statement"
txt = ["Here is my statement"]

In [76]:
test_result = Doc2VecModel2.docvecs.most_similar(positive=[Doc2VecModel2.infer_vector(txt)], topn=20)
test_result

[(32994, 0.824582040309906),
 (129672, 0.8166720271110535),
 (9339, 0.8056761026382446),
 (98444, 0.8023567199707031),
 (142480, 0.7966150045394897),
 (39591, 0.7949002981185913),
 (25758, 0.7909635305404663),
 (87757, 0.7891994714736938),
 (131735, 0.787695050239563),
 (127391, 0.7850116491317749),
 (137566, 0.775773823261261),
 (56603, 0.753994882106781),
 (141823, 0.7537686228752136),
 (110967, 0.7531044483184814),
 (27541, 0.7526343464851379),
 (138084, 0.7512686252593994),
 (140584, 0.7506092190742493),
 (8844, 0.7468898296356201),
 (34384, 0.7423561215400696),
 (76398, 0.7412362098693848)]